In [1]:
import numpy as np
import pandas as pd
import shutil

In [2]:
test = list(np.load('./patient_list/all_test_img_files.npy'))
val = list(np.load('./patient_list/all_val_img_files.npy'))

gbm_test = list(np.load('./patient_list/gbm_test_img_files.npy'))
gbm_val = list(np.load('./patient_list/gbm_val_img_files.npy'))

lgg_test = list(np.load('./patient_list/lgg_test_img_files.npy'))
lgg_val = list(np.load('./patient_list/lgg_val_img_files.npy'))

In [15]:
TEST_IMG_PATH = './test_images/'
TEST_LAB_PATH = './val_labels/'

GBM_TEST_IMG_PATH = './GBM_test_images/'
GBM_TEST_LAB_PATH = './GBM_test_labels/'

LGG_TEST_IMG_PATH = './LGG_test_images/'
LGG_TEST_LAB_PATH = './LGG_test_labels/'

In [17]:
mypath=TEST_IMG_PATH

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [18]:
Counter(onlyfiles) == Counter(test)

True

In [5]:
for IMG_FILENAME in val:
    LABEL_FILENAME = IMG_FILENAME.split('_')[0]+'_label.npy'
    if IMG_FILENAME in lgg_val:
        IMG = LGG_TEST_IMG_PATH
        LAB = LGG_TEST_LAB_PATH
    elif IMG_FILENAME in gbm_val:
        IMG = GBM_TEST_IMG_PATH
        LAB = GBM_TEST_LAB_PATH
        
    shutil.copy(IMG+IMG_FILENAME, TEST_IMG_PATH+IMG_FILENAME)
    shutil.copy(LAB+LABEL_FILENAME, TEST_LAB_PATH+LABEL_FILENAME)

In [10]:
from collections import Counter

In [19]:
shutil.make_archive("test_images", "zip", "./test_images")

KeyboardInterrupt: 